In [1]:
import pandas as pd
import numpy as np
import math

In [91]:
def append_df_to_excel(filename, df, sheet_name='Sheet1', startrow=None,
                       truncate_sheet=False, 
                       **to_excel_kwargs):
    """
    Append a DataFrame [df] to existing Excel file [filename]
    into [sheet_name] Sheet.
    If [filename] doesn't exist, then this function will create it.

    Parameters:
      filename : File path or existing ExcelWriter
                 (Example: '/path/to/file.xlsx')
      df : dataframe to save to workbook
      sheet_name : Name of sheet which will contain DataFrame.
                   (default: 'Sheet1')
      startrow : upper left cell row to dump data frame.
                 Per default (startrow=None) calculate the last row
                 in the existing DF and write to the next row...
      truncate_sheet : truncate (remove and recreate) [sheet_name]
                       before writing DataFrame to Excel file
      to_excel_kwargs : arguments which will be passed to `DataFrame.to_excel()`
                        [can be dictionary]

    Returns: None
    """
    from openpyxl import load_workbook

    import pandas as pd

    # ignore [engine] parameter if it was passed
    if 'engine' in to_excel_kwargs:
        to_excel_kwargs.pop('engine')

    writer = pd.ExcelWriter(filename, engine='openpyxl')

    # Python 2.x: define [FileNotFoundError] exception if it doesn't exist 
    try:
        FileNotFoundError
    except NameError:
        FileNotFoundError = IOError


    try:
        # try to open an existing workbook
        writer.book = load_workbook(filename)

        # get the last row in the existing Excel sheet
        # if it was not specified explicitly
        if startrow is None and sheet_name in writer.book.sheetnames:
            startrow = writer.book[sheet_name].max_row

        # truncate sheet
        if truncate_sheet and sheet_name in writer.book.sheetnames:
            # index of [sheet_name] sheet
            idx = writer.book.sheetnames.index(sheet_name)
            # remove [sheet_name]
            writer.book.remove(writer.book.worksheets[idx])
            # create an empty sheet [sheet_name] using old index
            writer.book.create_sheet(sheet_name, idx)

        # copy existing sheets
        writer.sheets = {ws.title:ws for ws in writer.book.worksheets}
    except FileNotFoundError:
        # file does not exist yet, we will create it
        pass

    if startrow is None:
        startrow = 0

    # write out the new sheet
    df.to_excel(writer, sheet_name, startrow=startrow, **to_excel_kwargs)

    # save the workbook
    writer.save()
    
    
def updateValExcel(sheet_name,df,index_set,index_r,col,value):
    df.set_index(index_set, inplace=True)
    df[col][index_r] = value
    df.reset_index(level=0, inplace=True)
    updatedftoexcel(df,sheet_name)
    
def updatedftoexcel(df,sheetName):
    append_df_to_excel('Inventory optimization working.xlsx', df, sheet_name=sheetName,startrow=0,index=False)           


def doiCal(val,list_):
#     print('val',val)
#     print('list_',len(list_))
    list_ = [0 if math.isnan(x) else x for x in list_]
#     list_ = list_.to_list()
    doi = 0
    if val > 0:
        for i in list_ :
            val = val - i
            if val < 0:
                break
            else :
                doi = doi + 1
#     print('DOI--',doi)
    return doi
    
def doiUpdate(ed):
    for i in range(len(ed)):
        for j in types:
            for k in plants:
                if ed['OH'+j+k][i] != '' :
                    doi = doiCal(ed['OH'+j+k][i],ed['R'+j+k][i+1:])
                    ed['DOI'+j+k][i] = doi
    return ed  

In [92]:
# doiUpdate(ed['OHCP3'][8],ed['RCP3'][8+1:])
# ed['OHCP3'][5],ed['RCP3'][5+1:]

In [93]:
%%time
ex = pd.ExcelFile('Inventory optimization working.xlsx')
ed = ex.parse('Sheet2')


plants = ['P1','P2','P3','P4','P5','P6','P7','P8','P9','P10']
types = ['C','L','S']
table_type = ['R','OH','DOI']

RCP = ed[['RCP1', 'RCP2', 'RCP3', 'RCP4', 'RCP5', 'RCP6', 'RCP7', 'RCP8', 'RCP9','RCP10']]
RLP = ed[['RLP1', 'RLP2', 'RLP3', 'RLP4', 'RLP5', 'RLP6', 'RLP7', 'RLP8', 'RLP9','RLP10']]
RSP = ed[['RSP1', 'RSP2', 'RSP3', 'RSP4', 'RSP5', 'RSP6', 'RSP7', 'RSP8', 'RSP9','RSP10']]
ed.columns[2:]



DOICP = ed[['id','OHCP1', 'OHCP2', 'OHCP3', 'OHCP4', 'OHCP5','OHCP6', 'OHCP7', 'OHCP8', 'OHCP9', 'OHCP10']]
DOILP = ed[['id','OHLP1', 'OHLP2', 'OHLP3','OHLP4', 'OHLP5', 'OHLP6', 'OHLP7', 'OHLP8', 'OHLP9', 'OHLP10']]
DOISP = ed[['id','OHSP1', 'OHSP2', 'OHSP3', 'OHSP4', 'OHSP5', 'OHSP6', 'OHSP7', 'OHSP8', 'OHSP9','OHSP10']]

## Calculating Aggregation
ed['RCAgg'] = ed['RCP1'].fillna(0) +ed['RCP2'].fillna(0)+ed['RCP3'].fillna(0)+ed['RCP4'].fillna(0)+ed['RCP5'].fillna(0)+ed['RCP6'].fillna(0)+ed['RCP7'].fillna(0)+ed['RCP8'].fillna(0)+ed['RCP9'].fillna(0)+ed['RCP10'].fillna(0)
ed['RLAgg'] = ed['RLP1'].fillna(0) +ed['RLP2'].fillna(0)+ed['RLP3'].fillna(0)+ed['RLP4'].fillna(0)+ed['RLP5'].fillna(0)+ed['RLP6'].fillna(0)+ed['RLP7'].fillna(0)+ed['RLP8'].fillna(0)+ed['RLP9'].fillna(0)+ed['RLP10'].fillna(0)
ed['RSAgg'] = ed['RSP1'].fillna(0) +ed['RSP2'].fillna(0)+ed['RSP3'].fillna(0)+ed['RSP4'].fillna(0)+ed['RSP5'].fillna(0)+ed['RSP6'].fillna(0)+ed['RSP7'].fillna(0)+ed['RSP8'].fillna(0)+ed['RSP9'].fillna(0)+ed['RSP10'].fillna(0)

##update DOI values
# ed = doiUpdate(ed)

##updating output to excel
# updatedftoexcel(ed,'Sheet1')


Wall time: 1.27 s


In [94]:
para = ex.parse('Parameters')

discharge = [i for i in para.iloc[2,1:].values]
thresh = [i for i in para.iloc[3,1:].values]
order_quantity = [i for i in para.iloc[4:,1].values]
order_quantity = order_quantity[0]
order_quantity

48000

In [59]:
# for x in range(len(thresh)):
for k in plants:
    print(k,thresh[int(k[1])])

P1 3
P2 3
P3 9
P4 14
P5 18
P6 18
P7 18
P8 22
P9 22
P10 3


In [95]:
def checkNegative(list_ag,loc,def_inventory):
    list_ag = [0 if math.isnan(x) else x for x in list_ag]
#     print(len(list_ag),len(list_ag)-len(list_ag[loc:]))
    list_ag = list_ag[loc:]
    return_loc = 0
    for i in range(len(list_ag)):
        if def_inventory - list_ag[i] >= 0 :
            def_inventory = def_inventory -list_ag[i]
            return_loc = i
#             print(i,list_ag[i])
        else :
#             print(i,def_inventory -list_ag[i])
            break 
    if return_loc > 0:
        return return_loc + loc



## Aggregation logic Without Balance

def flagCal(ed):
    def_inventory_g = order_quantity
    for j in types:
        temp = None
        for i in range(len(ed)):
            for k in plants:
                if ed['DOI'+j+k][i] == thresh[int(k[1])] :

                    if j == 'C':
                        print('Typej=',j,' Plantk=',k,' Rowi=',i)                 
                        ed['R'+j+'Flag'][i] = def_inventory_g 
                        m = i
                        while m <= len(ed) :
                            print('m',m)
                            list_ag = ed['R'+j+'Agg']#[m+1:]
                            a = checkNegative(list_ag,m,def_inventory_g)
                            print('a-----',a)
                            if a == None :
                                m = len(ed) + 1
                            else :  
                                ed['R'+j+'Flag'][a+1] = def_inventory_g
                                m = a + 1

                        temp = 'Found'
                        #print('break3-----------')
                        break
                    if j == 'L':
                        print('Typej=',j,' Plantk=',k,' Rowi=',i)                 
                        ed['R'+j+'Flag'][i] = def_inventory_g 
                        m = i
                        while m <= len(ed) :
                            print('m',m)
                            list_ag = ed['R'+j+'Agg']#[m+1:]
                            a = checkNegative(list_ag,m,def_inventory_g)
                            print('a-----',a)
                            if a == None :
                                m = len(ed) + 1
                            else :  
                                ed['R'+j+'Flag'][a+1] = def_inventory_g
                                m = a + 1
                        #flagCal(RLP,i,j,k)
                        temp = 'Found'
                        print('break3-----------')
                        break
                    if j == 'S':
                        print('Typej=',j,' Plantk=',k,' Rowi=',i)                 
                        ed['R'+j+'Flag'][i] = def_inventory_g 
                        m = i
                        while m <= len(ed) :
                            print('m',m)
                            list_ag = ed['R'+j+'Agg']#[m+1:]
                            a = checkNegative(list_ag,m,def_inventory_g)
                            print('a-----',a)
                            if a == None :
                                m = len(ed) + 1
                            else :  
                                ed['R'+j+'Flag'][a+1] = def_inventory_g
                                m = a + 1
                        #flagCal(RSP,i,j,k)
                        temp = 'Found'
                        print('break3-----------')
                        break

            if temp:
                #print('break2')
                break
    #     if temp:
    #         #print('break2')
    #         break
    return ed

# ed = flagCal(ed)

In [96]:
%%time
## Filling DOI in dataframe
#ed = doiUpdate(ed)

## Fillin g 48000
ed = flagCal(ed)

#updatedftoexcel(ed,'Sheet1')

Typej= C  Plantk= P4  Rowi= 5
m 5
a----- 27
m 28
a----- 48
m 49
a----- 68
m 69
a----- 87
m 88
a----- 107
m 108
a----- 127
m 128
a----- 146
m 147
a----- 165


C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


m 166
a----- 183
m 184
a----- 201
m 202
a----- 219
m 220
a----- 238
m 239
a----- 259
m 260
a----- 281
m 282
a----- 303
m 304
a----- 323
m 324
a----- 344
m 345
a----- 365
m 366
a----- None
Typej= L  Plantk= P1  Rowi= 2
m 2
a----- 10
m 11
a----- 19
m 20
a----- 28
m 29
a----- 36
m 37
a----- 44
m 45
a----- 52


C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


m 53
a----- 60
m 61
a----- 68
m 69
a----- 76
m 77
a----- 84
m 85
a----- 92
m 93
a----- 99
m 100
a----- 106
m 107
a----- 113
m 114
a----- 120
m 121
a----- 128
m 129
a----- 136
m 137
a----- 144
m 145
a----- 152
m 153
a----- 160
m 161
a----- 168
m 169
a----- 176
m 177
a----- 184
m 185
a----- 192
m 193
a----- 200
m 201
a----- 208
m 209
a----- 216
m 217
a----- 224
m 225
a----- 232
m 233
a----- 240
m 241
a----- 247
m 248
a----- 254
m 255
a----- 261
m 262
a----- 268
m 269
a----- 275
m 276
a----- 283
m 284
a----- 291
m 292
a----- 299
m 300
a----- 306
m 307
a----- 313
m 314
a----- 320
m 321
a----- 327
m 328
a----- 334
m 335
a----- 341
m 342
a----- 348
m 349
a----- 355
m 356
a----- 362
m 363
a----- 365
m 366
a----- None
break3-----------
Typej= S  Plantk= P1  Rowi= 4
m 4
a----- 174
m 175
a----- 347
m 348
a----- 365
m 366
a----- None
break3-----------
Wall time: 2.49 s


C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [31]:
ed1 = ed

In [44]:
ed = ed1

366

In [97]:
#######
def contriPlant(df,type_,start,end):
    list_a = []
    for k in plants:
        list_a.append(df['R'+type_+k][start:end].fillna(0).sum())
#         print('Plant:',k,'--',df['R'+type_+k][start:end].fillna(0).sum())
    if list_a:
        return list_a
    else :
        return None
# list_a = contriPlant(ed,'C',3,8)
# print(list_a)


#########  def Actual Flag Value


def checkNegative1(list_ag,loc,def_inventory):
    list_ag = [0 if math.isnan(x) else x for x in list_ag]
#     print(len(list_ag),len(list_ag)-len(list_ag[loc:]))
    list_ag = list_ag[loc:]
    return_loc = 0
    for i in range(len(list_ag)):
        if def_inventory - list_ag[i] >= 0 :
            def_inventory = def_inventory -list_ag[i]
            return_loc = i
#             print(i,list_ag[i])
        else :
#             print(i,def_inventory -list_ag[i])
            break 
    if return_loc > 0:
#         list_ = contriPlant(df,j,loc,return_loc + loc+1)
        return return_loc + loc




def actualReceived(ed):
    def_inventory_g = order_quantity
    for j in types:
        temp = None
#         for i in range(len(ed)):
        for i in range(len(ed)-1,-1,-1):
            for k in plants:
                if ed['R'+j+'ActFlag'][i] > 0 :
                    if j == 'C':
                        list_ag = ed['R'+j+'Agg']
                        print('Value',ed['R'+j+'ActFlag'][i],'Typej=',j,' Plantk=',k,' Rowi=',i)                 

    #                     ed['R'+j+'ActFlag'][i] = def_inventory_g 

                        t = checkNegative1(list_ag,i,ed['R'+j+'ActFlag'][i])
                        list_ = contriPlant(ed,j,i,t+1)
                        for x in range(len(plants)):
                            discharge_days = discharge[x]
                            ed['R'+j+'Act'+plants[x]][i+discharge_days] = list_[x] 

                        print('list_---- Once ',list_)


    #                     ed['R'+j+'Act'+k][i] = ed['R'+j+k][i:t].fillna(0).sum()

                        ed['R'+j+'ActFlag'][t+1] =  def_inventory_g
                        m = t + 1
                        while m <= len(ed):
                            print('m',m)
                            #[m+1:]

                            a = checkNegative1(list_ag,m,def_inventory_g)

    #                         a ,list_ = checkNegative1(ed,j,list_ag,m,def_inventory_g)

                            print('a-----',a)
                            if a == None :
                                m = len(ed) + 1
                            else : 
#                                 list_ = contriPlant(ed,j,m,a+1)
#                                 for x in range(len(plants)):
#                                     ed['R'+j+'Act'+plants[x]][m] = list_[x] 
                                ed['R'+j+'ActFlag'][a+1] = def_inventory_g
                                m = a + 1

                        temp = 'Found'
                        #print('break3-----------')
#                         print('**********','R'+j+'Flag',len(ed['R'+j+'Flag'][i+1:len(ed)]),'R'+j+'ActFlag',len(ed['R'+j+'ActFlag'][i+1:len(ed)]))
                        ed['R'+j+'Flag'][i+1:len(ed)] = ed['R'+j+'ActFlag'][i+1:len(ed)]
                        temp1 = ed['R'+j+'ActFlag'][0:i+1] 
                        ed['R'+j+'ActFlag'] = np.NAN
                        ed['R'+j+'ActFlag'] = temp1
                        break

                    if j == 'L':
                        list_ag = ed['R'+j+'Agg']
                        print('Value',ed['R'+j+'ActFlag'][i],'Typej=',j,' Plantk=',k,' Rowi=',i)                 

    #                     ed['R'+j+'ActFlag'][i] = def_inventory_g 

                        t = checkNegative1(list_ag,i,ed['R'+j+'ActFlag'][i])
                        list_ = contriPlant(ed,j,i,t+1)
                        for x in range(len(plants)):
                            discharge_days = discharge[x]
                            ed['R'+j+'Act'+plants[x]][i+discharge_days] = list_[x] 

                        print('list_---- Once ',list_)


    #                     ed['R'+j+'Act'+k][i] = ed['R'+j+k][i:t].fillna(0).sum()

                        ed['R'+j+'ActFlag'][t+1] =  def_inventory_g
                        m = t + 1
                        while m <= len(ed):
                            print('m',m)
                            #[m+1:]

                            a = checkNegative1(list_ag,m,def_inventory_g)

    #                         a ,list_ = checkNegative1(ed,j,list_ag,m,def_inventory_g)

                            print('a-----',a)
                            if a == None :
                                m = len(ed) + 1
                            else : 
#                                 list_ = contriPlant(ed,j,m,a+1)
#                                 for x in range(len(plants)):
#                                     ed['R'+j+'Act'+plants[x]][m] = list_[x] 
                                ed['R'+j+'ActFlag'][a+1] = def_inventory_g
                                m = a + 1

                        temp = 'Found'
                        #print('break3-----------')
#                         print('**********','R'+j+'Flag',len(ed['R'+j+'Flag'][i+1:len(ed)]),'R'+j+'ActFlag',len(ed['R'+j+'ActFlag'][i+1:len(ed)]))
                        ed['R'+j+'Flag'][i+1:len(ed)] = ed['R'+j+'ActFlag'][i+1:len(ed)]
                        temp1 = ed['R'+j+'ActFlag'][0:i+1] 
                        ed['R'+j+'ActFlag'] = np.NAN
                        ed['R'+j+'ActFlag'] = temp1
                        break

                    if j == 'S':
                        list_ag = ed['R'+j+'Agg']
                        print('Value',ed['R'+j+'ActFlag'][i],'Typej=',j,' Plantk=',k,' Rowi=',i)                 

    #                     ed['R'+j+'ActFlag'][i] = def_inventory_g 

                        t = checkNegative1(list_ag,i,ed['R'+j+'ActFlag'][i])
                        list_ = contriPlant(ed,j,i,t+1)
                        for x in range(len(plants)):
                            discharge_days = discharge[x]
                            ed['R'+j+'Act'+plants[x]][i+discharge_days] = list_[x] 

                        print('list_---- Once ',list_)


    #                     ed['R'+j+'Act'+k][i] = ed['R'+j+k][i:t].fillna(0).sum()

                        ed['R'+j+'ActFlag'][t+1] =  def_inventory_g
                        m = t + 1
                        while m <= len(ed):
                            print('m',m)
                            #[m+1:]

                            a = checkNegative1(list_ag,m,def_inventory_g)

    #                         a ,list_ = checkNegative1(ed,j,list_ag,m,def_inventory_g)

                            print('a-----',a)
                            if a == None :
                                m = len(ed) + 1
                            else : 
#                                 list_ = contriPlant(ed,j,m,a+1)
#                                 for x in range(len(plants)):
#                                     ed['R'+j+'Act'+plants[x]][m] = list_[x] 
                                ed['R'+j+'ActFlag'][a+1] = def_inventory_g
                                m = a + 1

                        temp = 'Found'
                        #print('break3-----------')
#                         print('**********','R'+j+'Flag',len(ed['R'+j+'Flag'][i+1:len(ed)]),'R'+j+'ActFlag',len(ed['R'+j+'ActFlag'][i+1:len(ed)]))
                        ed['R'+j+'Flag'][i+1:len(ed)] = ed['R'+j+'ActFlag'][i+1:len(ed)]
                        temp1 = ed['R'+j+'ActFlag'][0:i+1] 
                        ed['R'+j+'ActFlag'] = np.NAN
                        ed['R'+j+'ActFlag'] = temp1
                        break



            if temp:
                #print('break2')
                break
    #     if temp:
    #         #print('break2')
    #         break

    return ed




In [98]:
# len(ed['RCFlag']),len(ed['RCActFlag'])
# ed['RCActFlag'] = 0

In [99]:
# ed['RCFlag'][1:10] = ed['RCActFlag'][1:10]
# ed['RCFlag'][1:10]
# (ed['RCActFlag'][360:len(ed)]),(ed['RCFlag'][360:len(ed)])


In [100]:
%%time 
ed = actualReceived(ed)
# ed.head()

Value 40000 Typej= C  Plantk= P1  Rowi= 1


C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


list_---- Once  [5951.206146608766, 4592.490157154673, 0, 1878.568031430935, 3769.011013751034, 5883.8709677419365, 4592.490157154673, 3800.0, 3983.8709677419342, 4106.4516129032245]
m 20
a----- 41
m 42
a----- 61
m 62
a----- 80
m 81
a----- 100
m 101
a----- 120
m 121
a----- 139
m 140
a----- 158
m 159
a----- 176
m 177
a----- 194
m 195
a----- 212
m 213
a----- 231
m 232
a----- 251
m 252
a----- 272
m 273
a----- 294
m 295
a----- 315
m 316
a----- 335
m 336
a----- 356
m 357
a----- 365
m 366
a----- None
Value 50000 Typej= L  Plantk= P1  Rowi= 1


C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


list_---- Once  [4527.015516439205, 7131.227791563275, 2080.887096774194, 6268.855459057071, 4340.438453784119, 4645.1612903225805, 6967.74193548387, 4064.5161290322576, 3483.870967741935, 3774.1935483870975]
m 10
a----- 18
m 19
a----- 27
m 28
a----- 35
m 36
a----- 43
m 44
a----- 51
m 52
a----- 59
m 60
a----- 67


C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


m 68
a----- 75
m 76
a----- 83
m 84
a----- 91
m 92
a----- 98
m 99
a----- 105
m 106
a----- 112
m 113
a----- 119
m 120
a----- 127
m 128
a----- 135
m 136
a----- 143
m 144
a----- 151
m 152
a----- 159
m 160
a----- 167
m 168
a----- 175
m 176
a----- 183
m 184
a----- 191
m 192
a----- 199
m 200
a----- 207
m 208
a----- 215
m 216
a----- 223
m 224
a----- 231
m 232
a----- 239
m 240
a----- 247
m 248
a----- 254
m 255
a----- 261
m 262
a----- 268
m 269
a----- 275
m 276
a----- 283
m 284
a----- 291
m 292
a----- 299
m 300
a----- 306
m 307
a----- 313
m 314
a----- 320
m 321
a----- 327
m 328
a----- 334
m 335
a----- 341
m 342
a----- 348
m 349
a----- 355
m 356
a----- 362
m 363
a----- 365
m 366
a----- None


C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:148: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Value 50000 Typej= S  Plantk= P1  Rowi= 1
list_---- Once  [24309.23076923075, 923.076923076923, 0, 0, 0, 23838.461538461495, 923.076923076923, 0, 0, 0]
m 180
a----- 352
m 353
a----- 365
m 366
a----- None
Wall time: 3.59 s


C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:178: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [78]:
ed['RCActP1']

0         0
1      5951
2         0
3         0
4         0
5         0
6         0
7         0
8         0
9         0
10        0
11        0
12        0
13        0
14        0
15        0
16        0
17        0
18        0
19        0
20        0
21        0
22        0
23        0
24        0
25        0
26        0
27        0
28        0
29        0
       ... 
336       0
337       0
338       0
339       0
340       0
341       0
342       0
343       0
344       0
345       0
346       0
347       0
348       0
349       0
350       0
351       0
352       0
353       0
354       0
355       0
356       0
357       0
358       0
359       0
360       0
361       0
362       0
363       0
364       0
365       0
Name: RCActP1, dtype: int64

In [166]:
ed.to_csv('test.csv')

In [101]:
#updating output to excel
updatedftoexcel(ed,'Sheet1')

In [102]:

# ed[['RCActFlag','RCActP1','RCActP2','RCActP3']]
# ed['RCActP1']

In [ ]:
## Not using so far
## Aggregation logic With Balance


def_inventory_g = 48000
def_inventory = 48000

for j in types:
    temp = None
    for i in range(len(ed)):
        for k in plants:
            if ed['DOI'+j+k][i] == 3 :
                               
                if j == 'C':
                    print('Typej=',j,' Plantk=',k,' Rowi=',i)
#                     m = i 
#                     while 
#                     ed['R'+j+'Flag'][i] = def_inventory_g
                    
                    ed['R'+j+'Flag'][i] = def_inventory_g
#                     print('RCP',k[1],np.sum(RCP.iloc[i:i+1,int(k[1])-1:].fillna(0).values))
                    def_inventory = def_inventory_g - np.sum(RCP.iloc[i:i+1,int(k[1])-1:].fillna(0).values)
                    m = i
                    m_ = i
                    while m_ <= len(ed) :
                        print('m+1',m+1,'def_inventory',def_inventory)
                        list_ag = ed['R'+j+'Agg']#[m+1:]
                        a,b = checkNegative(list_ag,m+1,def_inventory)
#                         print('a,b',m+1+a,b)
                        if a == None :
                            m = len(ed) + 1
                        else :   
                            ed['R'+j+'Flag'][m+a] = def_inventory_g
                            m_ = m_ + m +a
                            m= a
                            def_inventory = def_inventory_g + b
#                             print(m+1,a,b,def_inventory)
                        
                    temp = 'Found'
                    #print('break3-----------')
                    break
                if j == 'L':
                    #flagCal(RLP,i,j,k)
                    temp = 'Found'
                    print('break3-----------')
                    break
                if j == 'S':
                    #flagCal(RSP,i,j,k)
                    temp = 'Found'
                    print('break3-----------')
                    break

        if temp:
            #print('break2')
            break
#     if temp:
#         #print('break2')
#         break


In [ ]:
def checkIfNegetive(number,lofl):
    for idx,i in enumerate(lofl):
        
        if i!=np.NAN:
#             print('Not nege',number,i,number-i)
            if number-i>=0:
                number=number-i
#                 print('Negative No.',number)
#                 lofl[idx]=9999
            else:
                return idx, number-i
    return False,number



def flagCal(df,initial_row,initial_col,number):
#     initial_col=7
#     number=48000
    for idx,i in enumerate(df[initial_row:].values):
        if idx==0:
            i=i[initial_col:]

        flag,number=checkIfNegetive(number,i)
        #print(idx,flag,number)
        if flag!=False:
#             print(flag,idx)
            return flag,idx+initial_row,number
            break

In [ ]:
RCP=RCP.fillna(0)
val_r,val_c,number = flagCal(RCP,2,7,48000)
val_r,val_c,number

In [ ]:

def_inventory = 48000
for j in types:
    temp = None
    for i in range(len(ed)):
        for k in plants:
            if ed['DOI'+j+k][i] == 3 :
                print('value-',ed['DOI'+j+k][i],'j-',j,'k-',k,'i-',i)
                
                ed['R'+j+'Flag'][i] = def_inventory
                if j == 'C':
                    m = i
                    while m <= len(ed):
                        val = None
                        RCP=RCP.fillna(0)
                        val = flagCal(RCP,m,int(k[1]),def_inventory)
                        print(val)
                        
                        if val :
                            ed['R'+j+'Flag'][val+1] = def_inventory
                    temp = 'Found'
                    #print('break3-----------')
                    break
                if j == 'L':
                    #flagCal(RLP,i,j,k)
                    temp = 'Found'
                    #print('break3-----------')
                    break
                if j == 'S':
                    #flagCal(RSP,i,j,k)
                    temp = 'Found'
                    #print('break3-----------')
                    break

        if temp:
            #print('break2')
            break
#     if temp:
#         #print('break2')
#         break

In [127]:
#updating output to excel
updatedftoexcel(ed,'Sheet1')

#updateValExcel('Emergency Demand (Tonnes) - C',ed,'Plant',ed_ordered['Plant'][i],'O-'+ed_ordered['Type'][i],ed_ordered['Suggestion'][i])
